<a href="https://colab.research.google.com/github/entmike/disco-diffusion-1/blob/main/Simplified_Disco_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simplified Disco Diffusion

🙏 Inspired from [alembics Notebook](https://colab.research.google.com/github/alembics/disco-diffusion/blob/main/Disco_Diffusion.ipynb) and others.

🙋‍♂️ Contributions welcomed at [https://github.com/entmike/disco-diffusion-1](https://github.com/entmike/disco-diffusion-1)

## Changes/Enhancements

- All functions moved to `dd.py` that are not needed in the Notebook to reduce clutter and hopefully improve readibility.

- Command-Line Support!
  After running the **Set Up Environment** cell, from your Google Colab Terminal you can run your Disco Diffusion workload from a terminal or make a `bash` script to do multiple different batches.  Example:

  ```bash
  cd /content/gdrive/MyDrive/disco-diffusion-1
  python disco.py --steps=50 --batch_name="CommandLineBatch" --RN50=False \
  --text_prompts='{"0":["A beautiful painting of a dolphin","ocean theme"]}'
  ```
- YAML Support from Terminal

  Use a YAML file to save/change your settings.  (See `examples/configs/lighthouse.yml` for an example structure.)
   ```bash
   cd /content/gdrive/MyDrive/disco-diffusion-1
   python disco.py --config_file=examples/configs/lighthouse.yml
   ```


In [6]:
import os, sys
import subprocess, torch

#@title Set Up Environment{ display-mode: "form" }
use_google_drive = True #@param {type:"boolean"}
save_models = True #@param {type:"boolean"}
content_root = '/content'
if use_google_drive == True:
  import os
  from google.colab import drive
  if os.path.isdir('/content/gdrive') == False:
    print(f'📁 Mounting Google Drive.  Please accept any confirmation screens.')
    drive.mount('/content/gdrive/')
  else:
    print(f'📁 Google Drive already mounted.')
  content_root = '/content/gdrive/MyDrive'

dd_root = f'{content_root}/disco-diffusion-1'
print(f'✅ Disco Diffusion root path will be "{dd_root}"')

root_path = dd_root

#@title Environment Setup { vertical-output: true, display-mode: "form" }
repo = 'https://github.com/entmike/disco-diffusion-1'
print(f'📦 Upgrading pyyaml...')
!pip install --upgrade pyyaml --quiet # &> /dev/null

if not os.path.isdir(f'{dd_root}') == False:
  os.chdir(f'{content_root}/..')
  subprocess.run(f'git clone {repo}'.split(' '), stdout=subprocess.PIPE).stdout.decode("utf-8")

os.chdir(f'{dd_root}/..')
print(f'📄 Pulling updates from GitHub...')
subprocess.run(f'git clean -df && git checkout -- . && git pull'.split(' '), stdout=subprocess.PIPE).stdout.decode("utf-8")
print(f'📦 Installing pip requirements...')
subprocess.run(f'pip install -r colab-requirements.txt'.split(' '), stdout=subprocess.PIPE).stdout.decode("utf-8")

from pydotted import pydot
PROJECT_DIR = dd_root
sys.path.append(f"{PROJECT_DIR}")
import dd, dd_args
model_path = f"{root_path}/models"
initDirPath = f"{root_path}/init_images"
outDirPath = f"{root_path}/images_out"
dd.createPath(model_path)
dd.createPath(initDirPath)
dd.createPath(outDirPath)
USE_ADABINS = True
TRANSLATION_SCALE = 1.0 / 200.0
MAX_ADABINS_AREA = 500000

simple_nvidia_smi_display = True #@param {type:"boolean"}
cuda_device = "cuda:0" #@param {type:"string"}
if simple_nvidia_smi_display:
    nvidiasmi_output = subprocess.run(["nvidia-smi", "-L"], stdout=subprocess.PIPE).stdout.decode("utf-8")
    print(f'🔎 {nvidiasmi_output}')
else:
    nvidiasmi_output = subprocess.run(["nvidia-smi"], stdout=subprocess.PIPE).stdout.decode("utf-8")
    print(nvidiasmi_output)
    nvidiasmi_ecc_note = subprocess.run(["nvidia-smi", "-i", "0"], stdout=subprocess.PIPE).stdout.decode("utf-8")
    print(nvidiasmi_ecc_note)

DEVICE = torch.device(cuda_device if torch.cuda.is_available() else "cpu")
device = DEVICE  # At least one of the modules expects this name..

print(f'✅ Using device: {DEVICE}')

try:
    # Fails if CPU is set
    if torch.cuda.get_device_capability(DEVICE) == (8, 0):  ## A100 fix thanks to Emad
        print("⚠️ Disabling CUDNN for A100 gpu", file=sys.stderr)
        torch.backends.cudnn.enabled = False
except:
    print("🛑 Are you using a CPU?  Check your PyTorch version if you get errors.")
    # torch.backends.cudnn.enabled = False
    pass

import wget
if save_models:
  # Download models if not present
  for m in [
      {
          "file": f"{model_path}/dpt_large-midas-2f21e586.pt",
          "url": "https://github.com/intel-isl/DPT/releases/download/1_0/dpt_large-midas-2f21e586.pt",
      },
      {
          "file": f"{model_path}/512x512_diffusion_uncond_finetune_008100.pt",
          "url": "https://v-diffusion.s3.us-west-2.amazonaws.com/512x512_diffusion_uncond_finetune_008100.pt",
      },
      {
          "file": f"{model_path}/256x256_diffusion_uncond.pt",
          "url": "https://openaipublic.blob.core.windows.net/diffusion/jul-2021/256x256_diffusion_uncond.pt",
      },
      {
          "file": f"{model_path}/secondary_model_imagenet_2.pth",
          "url": "https://v-diffusion.s3.us-west-2.amazonaws.com/secondary_model_imagenet_2.pth",
      },
      {
          "file": f"{PROJECT_DIR}/pretrained/AdaBins_nyu.pt",
          "url": "https://cloudflare-ipfs.com/ipfs/Qmd2mMnDLWePKmgfS8m6ntAg4nhV5VkUyAydYBp8cWWeB7/AdaBins_nyu.pt",
      },
  ]:
      if not os.path.exists(f'{m["file"]}'):
          print(f'🌍 (First time setup): Downloading model from {m["url"]} to {m["file"]}')
          wget.download(m["url"], m["file"])
      else:
        print(f'✅ Model already downloaded: {m["file"]}')

📁 Google Drive already mounted.
✅ Disco Diffusion root path will be "/content/gdrive/MyDrive/disco-diffusion-1"
📦 Upgrading pyyaml...
📄 Pulling updates from GitHub...
📦 Installing pip requirements...
🔎 GPU 0: Tesla T4 (UUID: GPU-41f3405b-38d3-e768-f0d2-2cdb0da7e56d)

✅ Using device: cuda:0
✅ Model already downloaded: /content/gdrive/MyDrive/disco-diffusion-1/models/dpt_large-midas-2f21e586.pt
✅ Model already downloaded: /content/gdrive/MyDrive/disco-diffusion-1/models/512x512_diffusion_uncond_finetune_008100.pt
✅ Model already downloaded: /content/gdrive/MyDrive/disco-diffusion-1/models/256x256_diffusion_uncond.pt
✅ Model already downloaded: /content/gdrive/MyDrive/disco-diffusion-1/models/secondary_model_imagenet_2.pth
✅ Model already downloaded: /content/gdrive/MyDrive/disco-diffusion-1/pretrained/AdaBins_nyu.pt


In [7]:
#@markdown ## Main Parameters
batch_name = "TimeToDisco" #@param {type:"string"}
set_seed = 'random_seed' #@param type="raw"
n_batches = 50 #@param {type:"number"}
width_height = [1280,768] #@param {type:"raw"}
steps = 170 #@param {type:"slider", min:0, max:1000, step:10}
text_prompts = {     0: ["A beautiful painting of a singular lighthouse, shining its light across a tumultuous sea of blood by greg rutkowski and thomas kinkade, Trending on artstation.", "yellow color scheme"],     100: ["This set of prompts start at frame 100","This prompt has weight five:5"], } #@param type="raw"
image_prompts = { } #@param type="raw"
google_drive = False
#@markdown ## Models
RN50 = True #@param {type:"boolean"}
RN101 = False #@param {type:"boolean"}
RN50x64 = False #@param {type:"boolean"}
RN50x16 = False #@param {type:"boolean"}
RN50x4 = False #@param {type:"boolean"}
ViTB16 = True #@param {type:"boolean"}
ViTB32 = True #@param {type:"boolean"}
ViTL14 = False #@param {type:"boolean"}
ViTL14_336 = False #@param {type:"boolean"}
use_secondary_model = True #@param {type:"boolean"}
diffusion_model = "512x512_diffusion_uncond_finetune_008100" #@param ["512x512_diffusion_uncond_finetune_008100", "256x256_diffusion_uncond"]
use_checkpoint = True #@param {type:"boolean"}
cutout_debug = False #@param {type:"boolean"}
diffusion_sampling_mode = "ddim" #@param ["ddim", "plms"]
clip_guidance_scale = 5000 #@param {type:"number"}
tv_scale = 0 #@param {type:"number"}
range_scale = 150 #@param {type:"number"}
sat_scale = 0 #@param {type:"number"}
cutn_batches = 4 #@param {type:"number"}
skip_augs = False #@param {type:"boolean"}
init_image = None #@param {type:"raw"}
init_scale = 1000 #@param {type:"number"}
skip_steps = 10 #@param {type:"number"}
animation_mode = "None" #@param ["None", "2D", "3D", "Video Input"]
video_init_path = "training.mp4" #@param {type:"string"}
extract_nth_frame = 10 #@param {type:"number"}
video_init_seed_continuity = True #@param {type:"boolean"}
key_frames = True #@param {type:"boolean"}
max_frames = 10000 #@param {type:"number"}
interp_spline = "Linear" #@param ["Linear", "Quadratic", "Cubic"]
resume_run = False #@param {type:"boolean"}
run_to_resume = "latest" #@param {type:"string"}
frames_skip_steps = "60%" #@param {type:"string"}
intermediate_saves = 0 #@param {type:"number"}
intermediates_in_subfolder = True #@param {type:"boolean"}
console_preview = False #@param {type:"boolean"}
console_preview_width = 80 #@param {type:"number"}
display_rate = 50 #@param {type:"number"}
retain_overwritten_frames = False #@param {type:"boolean"}
vr_mode = False #@param {type:"boolean"}
angle = '0:(0)' #@param {type:"string"}
zoom = '0:(1),10:(1.05)' #@param {type:"string"}
translation_x="0: (0)" #@param {type:"string"}
translation_y="0: (0)" #@param {type:"string"}
translation_z="0: (10.0)" #@param {type:"string"}
rotation_3d_x="0: (0)" #@param {type:"string"}
rotation_3d_y="0: (0)" #@param {type:"string"}
rotation_3d_z="0: (0)" #@param {type:"string"} 
midas_depth_model="dpt_large" #@param {type:"string"}
midas_weight=0.3 #@param {type:"number"}
near_plane = 0 #@param {type:"number"}
far_plane = 0 #@param {type:"number"}
fov = 0 #@param {type:"number"}
padding_mode = "border" #@param {type:"string"}
sampling_mode = "bicubic" #@param {type:"string"}
turbo_mode = False #@param {type:"boolean"}
turbo_steps=3 #@param {type:"number"}
turbo_preroll=10 #@param {type:"number"}
frames_scale=1500 #@param {type:"number"}
frames_skip_steps="60%" #@param {type:"string"}
cut_overview="[12]*400+[4]*600" #@param 
cut_innercut="[4]*400+[12]*600" #@param 
cut_icgray_p="[0.2]*400+[0]*600" #@param 
cut_ic_pow=1 #@param {type:"number"}
perlin_init=False #@param {type:"boolean"}
perlin_mode="mixed" #@param ["mixed", "color", "gray"]
eta=0.8 #@param {type:"number"}
clamp_grad=True #@param {type:"boolean"}
clamp_max=0.05 #@param {type:"number"}
randomize_class=True #@param {type:"boolean"}
clip_denoised=False #@param {type:"boolean"}
fuzzy_prompt=False #@param {type:"boolean"}
rand_mag=0.05 #@param {type:"number"}



videoFramesFolder = None
partialFolder = None
batchFolder = f"{outDirPath}/{batch_name}"
dd.createPath(f"{PROJECT_DIR}/pretrained")
dd.createPath(batchFolder)

In [ ]:
#@title Do Run
skip_video_for_run_all = True #@param {type:"boolean"}
from glob import glob
import os, random, math, gc, pathlib

# Get corrected sizes
side_x = (width_height[0] // 64) * 64
side_y = (width_height[1] // 64) * 64
if side_x != width_height[0] or side_y != width_height[1]:
    print(f"Changing output size to {side_x}x{side_y}. Dimensions must by multiples of 64.")

if animation_mode == "Video Input":
    videoFramesFolder = f"videoFrames"
    dd.createPath(videoFramesFolder)
    print(f"Exporting Video Frames (1 every {extract_nth_frame})...")
    max_frames = len(glob(f"{videoFramesFolder}/*.jpg"))
    try:
        for f in pathlib.Path(f"{videoFramesFolder}").glob("*.jpg"):
            f.unlink()
    except:
        print("")
    vf = f"select=not(mod(n\,{extract_nth_frame}))"
    subprocess.run(f'ffmpeg -i {video_init_path} -vf f{vf} -vsync -vfr -q:v 2 -loglevel error -stats {videoFramesFolder}/%04d.jpg'.split(" "), stdout=subprocess.PIPE).stdout.decode("utf-8")

# Insist turbo be used only w 3d anim.
if animation_mode != "3D" and (turbo_mode or vr_mode):
    print("⚠️ Turbo/VR modes only available with 3D animations. Disabling... ⚠️")
    turbo_mode = False
    vr_mode = False

if type(intermediate_saves) is not list:
    if intermediate_saves:
        steps_per_checkpoint = math.floor((steps - skip_steps - 1) // (intermediate_saves + 1))
        steps_per_checkpoint = steps_per_checkpoint if steps_per_checkpoint > 0 else 1
        print(f"Will save every {steps_per_checkpoint} steps")
    else:
        steps_per_checkpoint = steps + 10
else:
    steps_per_checkpoint = None

if intermediate_saves and intermediates_in_subfolder is True:
    partialFolder = f"{batchFolder}/partials"
    dd.createPath(partialFolder)

if retain_overwritten_frames is True:
    retainFolder = f"{batchFolder}/retained"
    dd.createPath(retainFolder)

if cutout_debug is True:
    cutoutDebugFolder = f"{batchFolder}/debug"
    dd.createPath(cutoutDebugFolder)

skip_step_ratio = int(frames_skip_steps.rstrip("%")) / 100
calc_frames_skip_steps = math.floor(steps * skip_step_ratio)

if steps <= calc_frames_skip_steps:
    sys.exit("⚠️ ERROR: You can't skip more steps than your total steps ⚠️")

if resume_run:
    if run_to_resume == "latest":
        try:
            batchNum  # type: ignore
        except:
            batchNum = len(glob(f"{batchFolder}/{batch_name}(*)_settings.txt")) - 1
    else:
        batchNum = int(run_to_resume)
    if resume_from_frame == "latest":
        start_frame = len(glob(batchFolder + f"/{batch_name}({batchNum})_*.png"))
        if animation_mode != "3D" and turbo_mode == True and start_frame > turbo_preroll and start_frame % int(turbo_steps) != 0:
            start_frame = start_frame - (start_frame % int(turbo_steps))
    else:
        start_frame = int(resume_from_frame) + 1
        if animation_mode != "3D" and turbo_mode == True and start_frame > turbo_preroll and start_frame % int(turbo_steps) != 0:
            start_frame = start_frame - (start_frame % int(turbo_steps))
        if retain_overwritten_frames is True:
            existing_frames = len(glob(batchFolder + f"/{batch_name}({batchNum})_*.png"))
            frames_to_save = existing_frames - start_frame
            print(f"Moving {frames_to_save} frames to the Retained folder")
            dd.move_files(
                start_frame,
                existing_frames,
                batchFolder,
                retainFolder,
                batch_name=batch_name,
                batchNum=batchNum,
            )
else:
    start_frame = 0
    batchNum = len(glob(batchFolder + "/*.txt"))
    while (
        os.path.isfile(f"{batchFolder}/{batch_name}({batchNum})_settings.txt") is True or os.path.isfile(f"{batchFolder}/{batch_name}-{batchNum}_settings.txt") is True
    ):
        batchNum += 1

if set_seed == "random_seed":
    random.seed()
    seed = random.randint(0, 2**32)
    print(f"🌱 Randomly using seed: {seed}")
else:
    seed = int(set_seed)

args = {
    "use_checkpoint": use_checkpoint,
    "cutout_debug": cutout_debug,
    "ViTB32": ViTB32,
    "ViTB16": ViTB16,
    "ViTL14": ViTL14,
    "ViTL14_336": ViTL14_336,
    "RN50": RN50,
    "RN50x4": RN50x4,
    "RN50x16": RN50x16,
    "RN50x64": RN50x64,
    "RN101": RN101,
    "diffusion_sampling_mode": diffusion_sampling_mode,
    "width_height": width_height,
    "clip_guidance_scale": clip_guidance_scale,
    "tv_scale": tv_scale,
    "range_scale": range_scale,
    "sat_scale": sat_scale,
    "cutn_batches": cutn_batches,
    "use_secondary_model": use_secondary_model,
    "diffusion_model": diffusion_model,
    "animation_mode": animation_mode,
    "batchNum": batchNum,
    "prompts_series": text_prompts,
    "text_prompts": text_prompts,
    "console_preview": console_preview,
    "console_preview_width": console_preview_width,
    "image_prompts_series": image_prompts,
    "seed": seed,
    "display_rate": display_rate,
    "n_batches": n_batches if animation_mode == "None" else 1,
    "batch_size": 1,
    "batch_name": batch_name,
    "steps": steps,
    "init_image": init_image,
    "init_scale": init_scale,
    "skip_steps": skip_steps,
    "side_x": side_x,
    "side_y": side_y,
    "animation_mode": animation_mode,
    "video_init_path": video_init_path,
    "extract_nth_frame": extract_nth_frame,
    "video_init_seed_continuity": video_init_seed_continuity,
    "key_frames": key_frames,
    "max_frames": max_frames if animation_mode != "None" else 1,
    "interp_spline": interp_spline,
    "start_frame": start_frame,
    "angle": angle,
    "zoom": zoom,
    "translation_x": translation_x,
    "translation_y": translation_y,
    "translation_z": translation_z,
    "rotation_3d_x": rotation_3d_x,
    "rotation_3d_y": rotation_3d_y,
    "rotation_3d_z": rotation_3d_z,
    "midas_depth_model": midas_depth_model,
    "midas_weight": midas_weight,
    "near_plane": near_plane,
    "far_plane": far_plane,
    "fov": fov,
    "padding_mode": padding_mode,
    "sampling_mode": sampling_mode,
    "frames_scale": frames_scale,
    "calc_frames_skip_steps": calc_frames_skip_steps,
    "skip_step_ratio": skip_step_ratio,
    "image_prompts": image_prompts,
    "cut_overview": cut_overview,
    "cut_innercut": cut_innercut,
    "cut_ic_pow": cut_ic_pow,
    "cut_icgray_p": cut_icgray_p,
    "intermediate_saves": intermediate_saves,
    "intermediates_in_subfolder": intermediates_in_subfolder,
    "steps_per_checkpoint": steps_per_checkpoint,
    "perlin_init": perlin_init,
    "perlin_mode": perlin_mode,
    "set_seed": set_seed,
    "eta": eta,
    "clamp_grad": clamp_grad,
    "clamp_max": clamp_max,
    "skip_augs": skip_augs,
    "randomize_class": randomize_class,
    "clip_denoised": clip_denoised,
    "fuzzy_prompt": fuzzy_prompt,
    "rand_mag": rand_mag,
    "turbo_mode": turbo_mode,
    "turbo_preroll": turbo_preroll,
    "turbo_steps": turbo_steps,
    "video_init_seed_continuity": video_init_seed_continuity,
    "videoFramesFolder": videoFramesFolder,
    "TRANSLATION_SCALE": TRANSLATION_SCALE,
    "partialFolder": partialFolder,
    "model_path": model_path,
    "batchFolder": batchFolder,
    "resume_run": resume_run,
}
# TODO: move model prep out of do run!!!!
args = pydot(args)  # Thx Zippy
# print(args)
try:
    dd.do_run(
        args=args,
        device=device,
        is_colab=True,
        batchNum=batchNum,
        start_frame=start_frame,
    )
except KeyboardInterrupt:
    print("🛑 Run interrupted by user.")
    pass
finally:
    gc.collect()
    torch.cuda.empty_cache()

if animation_mode != "None":
    if skip_video_for_run_all == True:
        print("⚠️ Skipping video creation, uncheck skip_video_for_run_all if you want to run it")
    else:
        dd.createVideo(args)


🌱 Randomly using seed: 405806767
{'use_checkpoint': True, 'cutout_debug': False, 'ViTB32': True, 'ViTB16': True, 'ViTL14': False, 'ViTL14_336': False, 'RN50': True, 'RN50x4': False, 'RN50x16': False, 'RN50x64': False, 'RN101': False, 'diffusion_sampling_mode': 'ddim', 'width_height': [1280, 768], 'clip_guidance_scale': 5000, 'tv_scale': 0, 'range_scale': 150, 'sat_scale': 0, 'cutn_batches': 4, 'use_secondary_model': True, 'diffusion_model': '512x512_diffusion_uncond_finetune_008100', 'animation_mode': 'None', 'batchNum': 4, 'prompts_series': {0: ['A beautiful painting of a singular lighthouse, shining its light across a tumultuous sea of blood by greg rutkowski and thomas kinkade, Trending on artstation.', 'yellow color scheme'], 100: ['This set of prompts start at frame 100', 'This prompt has weight five:5']}, 'text_prompts': {0: ['A beautiful painting of a singular lighthouse, shining its light across a tumultuous sea of blood by greg rutkowski and thomas kinkade, Trending on artstat